In [1]:
import pandas as pd 
import numpy as np 
import json 
import csv

# LOAD DATA

In [4]:
DPR_DATA='/PATH/dpr_scores.json'
EADPR_DATA='/PATH/eadpr_scores.json'

In [5]:
dpr_data = pd.read_json(DPR_DATA)
eadpr_data = pd.read_json(EADPR_DATA)

In [6]:
dpr_data.shape

(1383, 18)

In [2]:
import json
with open('/PATH/dpr/data/downloads/data/retriever/nq-train.json','r') as f:
    data = json.load(f)

# COLLECT FLIPPED DATA

In [7]:
dpr_filp_data = dpr_data[dpr_data['counterfactual_win']==1]
eadpr_filp_data = eadpr_data[eadpr_data['counterfactual_win']==1]

# Q TYPE CLASSIFICATION

Define question type

In [8]:
q_types = ['how','what','when','where','which','who','why']

Filtering dataframe for each question type

In [ ]:

# total number of samples for each q_type
q_type_samples={}

for q_type in q_types:
    dpr_filp_data[q_type]=dpr_filp_data['question'].str.contains(q_type)
    eadpr_filp_data[q_type]=eadpr_filp_data['question'].str.contains(q_type)

    dpr_data[q_type]=dpr_data['question'].str.contains(q_type)
    q_type_samples[q_type]=dpr_data[dpr_data[q_type]==True].shape[0]

dpr_avg_dict={} # passage score - counterfactual_score 
eadpr_avg_dict={}

# collect the number of flipped samples
dpr_flip={}
eadpr_flip={}

for q_type in q_types:
    # dpr -----------------
    # q_type filter
    df = dpr_filp_data[dpr_filp_data[q_type]==True]
    #count
    dpr_flip[q_type]=df.shape[0] 
    #score difference
    df['diff']=df['counterfactual_score']-df['passage_score']
    dpr_avg_dict[q_type]=df['diff'].mean()
    # eadpr -----------------
    # q_type filter
    df = eadpr_filp_data[eadpr_filp_data[q_type]==True]
    #count
    eadpr_flip[q_type]=df.shape[0]
     #score difference
    df['diff']=df['counterfactual_score']-df['passage_score']
    eadpr_avg_dict[q_type]=df['diff'].mean()


In [10]:

q_type_samples, sum(list(q_type_samples.values()))

({'how': 114,
  'what': 228,
  'when': 295,
  'where': 146,
  'which': 41,
  'who': 547,
  'why': 3},
 1374)

# DRAW A FIGURE

In [29]:
from matplotlib import pyplot as plt

In [ ]:
# plt setting

bar_types = {'one_bar':0,'two_bars':1}
BAR_TYPE='two_bars'

fig, ax = plt.subplots(figsize=(9,6))
xlabel, total = zip(*q_type_samples.items())
total = np.array(total)

bar_width = [1,0.7]
margin= [0,0.5]  
margin1=[0,0.1]
margin2=[0,0.9]

index = np.arange(0,len(dpr_flip)*2,2)

plt.ylabel('percentage')
plt.xlabel('question type (Natural Questions)')
plt.xticks(index+margin[bar_types[BAR_TYPE]], xlabel)

# draw
_, y = zip(*dpr_flip.items())
y = np.array(y)
y = y/total*100
plt.bar(index+margin1[bar_types[BAR_TYPE]],y,bar_width[bar_types[BAR_TYPE]],label='dpr', hatch='//',edgecolor='black',color='white')

_, y = zip(*eadpr_flip.items())
y = np.array(y)
y = y/total*100
plt.bar(index+margin2[bar_types[BAR_TYPE]],y,bar_width[bar_types[BAR_TYPE]],label='eadpr',edgecolor='#2b59b4',color='#d1e2ff')


plt.legend()
plt.savefig(f'AAR_qtype_inbatch_{BAR_TYPE}.png')

# Generate hard as pivot dataset with multiple BM25/dpr hard negs

In [43]:
pdf = pd.read_json('/PATH/nq-adv-hn-train-counterfactuals-sentence-mask-new-no-blank.json')

In [52]:
pdf_new=pdf.drop(columns=['cf_negative_ctxs'])
pdf_new=pdf_new[pdf_new['hard_negative_ctxs']!=pdf_new['negative_ctxs']]
pdf_new.head()

,question,answers,negative_ctxs,hard_negative_ctxs,positive_ctxs
0,big little lies season 2 how many episodes,[seven],[],"[{'id': '18768928', 'title': 'Big Little Lies ...","[{'title': 'Big Little Lies (TV series)', 'tex..."
1,who sang waiting for a girl like you,[Foreigner],[],"[{'id': '9601137', 'title': 'A Girl Like You (...","[{'title': 'Waiting for a Girl Like You', 'tex..."
2,where do you cross the arctic circle in norway,[Saltfjellet],[],"[{'id': '7484812', 'title': 'Nordkalottruta', ...","[{'title': 'Arctic Norway', 'text': 'Arctic No..."
3,who is the main character in green eggs and ham,[Sam - I - am],[],"[{'id': '7520524', 'title': 'The Green Man (Am...","[{'title': 'Green Eggs and Ham', 'text': '(in ..."
4,do veins carry blood to the heart or away,[to],[],"[{'id': '16063616', 'title': 'Venae cavae', 't...","[{'title': 'Vein', 'text': 'Vein Veins are blo..."


In [54]:
pdf_new.columns=['question','answers','negative_ctxs','cf_negative_ctxs','positive_ctxs']
pdf_new['hard_negative_ctxs']=pdf_new['negative_ctxs']
pdf_new.head()

,question,answers,negative_ctxs,cf_negative_ctxs,positive_ctxs,hard_negative_ctxs
0,big little lies season 2 how many episodes,[seven],[],"[{'id': '18768928', 'title': 'Big Little Lies ...","[{'title': 'Big Little Lies (TV series)', 'tex...",[]
1,who sang waiting for a girl like you,[Foreigner],[],"[{'id': '9601137', 'title': 'A Girl Like You (...","[{'title': 'Waiting for a Girl Like You', 'tex...",[]
2,where do you cross the arctic circle in norway,[Saltfjellet],[],"[{'id': '7484812', 'title': 'Nordkalottruta', ...","[{'title': 'Arctic Norway', 'text': 'Arctic No...",[]
3,who is the main character in green eggs and ham,[Sam - I - am],[],"[{'id': '7520524', 'title': 'The Green Man (Am...","[{'title': 'Green Eggs and Ham', 'text': '(in ...",[]
4,do veins carry blood to the heart or away,[to],[],"[{'id': '16063616', 'title': 'Venae cavae', 't...","[{'title': 'Vein', 'text': 'Vein Veins are blo...",[]


In [55]:
pdf_new.to_json('nq-adv-hn-train-all-hardnegs-as-counterfactuals.json',orient='records')